2025-11-28 08:00:19,691 INFO sqlalchemy.engine.Engine COMMIT
 * Debugger is active!
 * Debugger PIN: 153-414-109
2025-11-28 08:00:21,537 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-28 08:00:21,539 INFO sqlalchemy.engine.Engine SELECT products.id AS products_id, products.name AS products_name, products.category AS products_category, products.quantity AS products_quantity, products.unit AS products_unit, products."expirationDate" AS "products_expirationDate", products.supplier AS products_supplier, products.price AS products_price
FROM products
2025-11-28 08:00:21,539 INFO sqlalchemy.engine.Engine [generated in 0.00024s] ()
127.0.0.1 - - [28/Nov/2025 08:00:21] "GET /api/items HTTP/1.1" 500 -
Traceback (most recent call last):

Here is a **clear documentation** explaining *why the error happens*, *what caused it*, and *how to fix it properly*.

I made this documentation beginner-friendly, but still complete.

---

# 📘 **Documentation: Fixing SQLAlchemy Database Error in `/api/items`**

## ✅ **1. What Happened**

Your Flask API made a request:

```
GET /api/items
```

Inside this request, SQLAlchemy executed:

```sql
SELECT id, name, category, quantity, unit, expirationDate, supplier, price FROM products;
```

But the query crashed with:

```
500 Internal Server Error
```

Then SQLAlchemy printed the **table structure** it's trying to create:

```sql
CREATE TABLE "products" (
    "id" VARCHAR NOT NULL,
    "name" VARCHAR,
    "category" VARCHAR,
    "quantity" INTEGER,
    "unit" INTEGER,
    "expirationDate" VARCHAR,
    "supplier" VARCHAR,
    "price" FLOAT,
    PRIMARY KEY("id")
);
```

---

# ❗ **2. Root Cause**

### Your `Product` model is **different** from what SQLAlchemy expects.

Specifically:

### ❌ You removed columns from your model

but your database still has **old table structure** from before.

Example:

Your sample JSON contains:

* reorderLevel
* batchNumber
* sku

But the table above **does not include** those fields.

Your updated model likely looks like this:

```python
class Product(Base):
    __tablename__ = "products"

    id = Column(String, primary_key=True)
    name = Column(String)
    category = Column(String)
    quantity = Column(Integer)
    unit = Column(String)      # <-- but DB has unit = INTEGER
    reorderLevel = Column(Integer)
    expirationDate = Column(String)
    batchNumber = Column(String)
    supplier = Column(String)
    price = Column(Float)
    sku = Column(String)
```

### 🔥 **Mismatch causes SQLAlchemy to crash**.

---

# ❗ **3. Second Cause: Wrong `unit` type**

Your database shows:

```
unit INTEGER
```

But your JSON uses:

```
"unit": "bottles"
```

And your model probably uses:

```python
unit = Column(String)
```

### 🚨 SQLAlchemy cannot convert "bottles" → integer

This triggers a 500 error.

---

# 🔧 **4. How to Fix It (Cleanest Solution)**

### ✔ Step 1 — Delete your old SQLite database

Because SQLite does NOT auto-update table structure.

Delete:

```
inventory.db
```

---

### ✔ Step 2 — Update your Product model to match your JSON

```python
# models.py
from sqlalchemy import Column, String, Integer, Float
from database import Base

class Product(Base):
    __tablename__ = "products"

    id = Column(String, primary_key=True)
    name = Column(String)
    category = Column(String)
    quantity = Column(Integer)
    unit = Column(String)               # <-- Correct
    reorderLevel = Column(Integer)
    expirationDate = Column(String)
    batchNumber = Column(String)
    supplier = Column(String)
    price = Column(Float)
    sku = Column(String)
```

---

### ✔ Step 3 — Recreate tables

Run:

```python
from database import engine
from models import Base

Base.metadata.create_all(engine)
```

This will regenerate:

```
inventory.db
products table with correct columns
```

---

### ✔ Step 4 — Insert your sample products again (optional)

```python
service = InventoryService()
service.add_product(sample_products[0])
service.add_product(sample_products[1])
```

---

# 📘 **5. Why You Must Delete the Old Database**

SQLite **does NOT** update or migrate tables.

Meaning:

If you change your model →
SQLite will **not** change the table →
Columns mismatch →
Errors happen.

Deleting `inventory.db` gives SQLAlchemy a fresh start.

---

# 📘 **6. Prevent Future Errors**

If you frequently change models → use a migration tool:

### Install Alembic:

```
pip install alembic
```

This lets you run:

```
alembic upgrade head
```

To automatically update your database schema.

---

# 🎉 Done!

If you want, I can also generate:

📄 **PDF documentation**
✔ Complete **working project example**
✔ A **Postman API collection** for your CRUD routes

Just tell me!
